In [35]:
from google.cloud import bigquery
import pandas as pd

In [36]:
client = bigquery.Client()

In [112]:
cadastro = client.query('select * from hackgetnet-team56.raw.receita_federal_cnpj').to_dataframe()

In [113]:
serasa = client.query('select * from hackgetnet-team56.raw.serasa_score').to_dataframe()

In [114]:
booking = client.query('select * from hackgetnet-team56.raw.booking_score').to_dataframe()

In [115]:
cadastro = cadastro[['id', 'municipio', 'atividade_principal_code', 'situacao', 'fantasia', 'nome']]
cadastro.sample(2)

,id,municipio,atividade_principal_code,situacao,fantasia,nome
7,687681000108,BRASILIA,55.10-8-01,ATIVA,B HOTEL,BRASILIA EMPREENDIMENTOS SERVICOS E PARTICIPAC...
4,91046284000960,PORTO ALEGRE,55.10-8-01,ATIVA,HOLIDAY INN PORTO ALEGRE,PLATAMON PARTICIPACOES E EMPREENDIMENTOS LTDA


In [116]:
serasa = serasa[['id', 'score', 'valor_divida']]
serasa = serasa.rename({'score': 'serasa_score'}, axis=1)
serasa.sample(2)

,id,serasa_score,valor_divida
2,687681000108,336,10052
0,37069853000190,725,0


In [117]:
booking = booking[['id', 'score', 'localizationScore']]
booking = booking.rename({'score': 'avaliacao'}, axis=1)
booking.sample(2)

,id,avaliacao,localizationScore
4,28047759000149,NaN,NaN
2,30364042000172,NaN,NaN


In [139]:
df = pd.merge(cadastro, serasa, how='left', on='id')
df = pd.merge(df, booking, how='left', on='id')

In [140]:
df['tipo_pousada'] = ((df['fantasia'].str.contains('POUSADA')) & (df['nome'].str.contains('POUSADA'))).astype(int)
df['tipo_hotel'] = ((df['tipo_pousada'] == 0) & (df['atividade_principal_code'] == '55.10-8-01')).astype(int)
df['tipo_hostel'] = ((df['tipo_pousada'] == 0) & (df['tipo_hotel'] == 0)).astype(int)
del df['nome'], df['fantasia'], df['atividade_principal_code']

In [141]:
# situacao cadastral dos estabelecimentos
df = df.loc[df['situacao'] == 'ATIVA']
del df['situacao']

In [142]:
# divida do estabelecimento
df['valor_divida'] = df['valor_divida'].fillna(0).astype(int)

In [143]:
# score do mercado
df['serasa_score'] = df['serasa_score'] / 1000
df['avaliacao'] = df['avaliacao'] / 10
df['localizationScore'] = df['localizationScore'] / 10

In [144]:
is_centro_economico = ['BRASILIA', 'FLORIANOPOLIS']
is_praia = ['RECIFE', 'FLORIANOPOLIS']
is_serra = ['GRAMADO']
df['regiao_centro_economico'] = df['municipio'].isin(is_centro_economico).astype(int)
df['regiao_praia'] = df['municipio'].isin(is_praia).astype(int)
df['regiao_serra'] = df['municipio'].isin(is_serra).astype(int)
del df['municipio']

In [145]:
df.index = df['id']
del df['id']

In [146]:
df

,serasa_score,valor_divida,avaliacao,localizationScore,tipo_pousada,tipo_hotel,tipo_hostel,regiao_centro_economico,regiao_praia,regiao_serra
id,,,,,,,,,,
91170555000137,NaN,0,NaN,NaN,1,0,0,0,0,1
30364042000172,NaN,0,NaN,NaN,1,0,0,0,0,1
315446000458,NaN,0,NaN,NaN,0,1,0,0,0,1
5618132000103,NaN,0,0.96,0.99,1,0,0,0,0,1
91046284000960,NaN,0,0.80,NaN,0,1,0,0,0,0
24995806000125,NaN,0,NaN,NaN,0,0,1,0,0,1
87003323000117,NaN,0,0.90,NaN,0,1,0,0,0,1
687681000108,0.336,10052,0.92,0.95,0,1,0,1,0,0
28047759000149,NaN,0,NaN,NaN,0,1,0,1,1,0


In [147]:
df[df['tipo_hotel'] == 1]

,serasa_score,valor_divida,avaliacao,localizationScore,tipo_pousada,tipo_hotel,tipo_hostel,regiao_centro_economico,regiao_praia,regiao_serra
id,,,,,,,,,,
315446000458,NaN,0,NaN,NaN,0,1,0,0,0,1
91046284000960,NaN,0,0.80,NaN,0,1,0,0,0,0
87003323000117,NaN,0,0.90,NaN,0,1,0,0,0,1
687681000108,0.336,10052,0.92,0.95,0,1,0,1,0,0
28047759000149,NaN,0,NaN,NaN,0,1,0,1,1,0
87344370000124,NaN,0,NaN,NaN,0,1,0,0,0,0
